# <B> Create Custom Docker Image </B>
* Container: codna_pytorch_p310

## AutoReload

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. For preprocessing

In [ ]:
import boto3
from utils.ecr import ecr_handler

In [ ]:
ecr = ecr_handler()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

### 1.1 Get base image uri

In [ ]:
import sagemaker

In [ ]:
base_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    version="2.1",
    region=region,
    image_scope="training",
    instance_type="ml.g5.xlarge"
)

print (f'base_image_uri: {base_image_uri}')

In [ ]:
%%writefile ./custom-docker/dockerfile-prep

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.1-gpu-py310

RUN pip install -U pip boto3 sagemaker

COPY requirements.txt /opt/ml/packages/
RUN pip install -r /opt/ml/packages/requirements.txt

# 기본 환경변수
ENV PYTHONUNBUFFERED=TRUE

# Processing Job 환경변수
ENV SM_INPUT_DIR=/opt/ml/processing/input
ENV SM_OUTPUT_DIR=/opt/ml/processing/output

In [ ]:
strRepositoryName="prep-docker-image"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "dockerfile-prep"
strDockerDir = "./custom-docker/"
strTag = "latest"

* **strAccountId**는 베이스 이미지의 account_id 사용 (**763104351884**.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.6.0-gpu-py312)

In [ ]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName=region, strAccountId="763104351884", no_cache=True)
strEcrRepositoryUri_prep = ecr.register_image_to_ecr(region, account_id, strRepositoryName, strTag)

print (f'strEcrRepositoryUri_prep: {strEcrRepositoryUri_prep}')

## 2. For training

### 2.1 Get base image uri

In [ ]:
base_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    version="2.1",
    region=region,
    image_scope="training",
    instance_type="ml.g5.xlarge"
)

print (f'base_image_uri: {base_image_uri}')

In [ ]:
%%writefile ./custom-docker/dockerfile-tr

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.1-gpu-py310

RUN pip install -U pip boto3 sagemaker

COPY requirements.txt /opt/ml/packages/
RUN pip install -r /opt/ml/packages/requirements.txt

# 기본 환경변수
ENV PYTHONUNBUFFERED=TRUE
ENV PATH="/opt/ml/code:${PATH}"
ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code

In [ ]:
strRepositoryName="tr-docker-image"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "dockerfile-tr"
strDockerDir = "./custom-docker/"
strTag = "latest"

* **strAccountId**는 베이스 이미지의 account_id 사용 (**763104351884**.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.6.0-gpu-py312)

In [ ]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName=region, strAccountId="763104351884", no_cache=True)
strEcrRepositoryUri_tr = ecr.register_image_to_ecr(region, account_id, strRepositoryName, strTag)

print (f'strEcrRepositoryUri_tr: {strEcrRepositoryUri_tr}')

## 3. For inference

### 3.1 Get base image uri

In [ ]:
base_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    version="2.1",
    region=region,
    image_scope="inference",
    instance_type="ml.g5.xlarge"
)

print (f'base_image_uri: {base_image_uri}')

In [ ]:
%%writefile ./custom-docker/dockerfile-inf

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:2.1-gpu-py310

RUN pip install -U pip boto3 sagemaker

COPY requirements.txt /opt/ml/packages/
RUN pip install -r /opt/ml/packages/requirements.txt

# 기본 환경변수
ENV PYTHONUNBUFFERED=TRUE
ENV PATH="/opt/ml/code:${PATH}"
ENV SAGEMAKER_MODEL_DIR=/opt/ml/model

In [ ]:
strRepositoryName="inf-docker-image"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerFile = "dockerfile-inf"
strDockerDir = "./custom-docker/"
strTag = "latest"

* **strAccountId**는 베이스 이미지의 account_id 사용 (**763104351884**.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:2.6.0-gpu-py312)

In [ ]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName=region, strAccountId="763104351884", no_cache=True)
strEcrRepositoryUri_inf = ecr.register_image_to_ecr(region, account_id, strRepositoryName, strTag)

print (f'strEcrRepositoryUri_inf: {strEcrRepositoryUri_inf}')

## 3. [Optional] AWS Systems Manager Parameter Store 를 이용한 파라미터 저장/활용
- [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **SSM**: "arn:aws:iam::aws:policy/AmazonSSMFullAccess"<BR>

In [ ]:
from utils.ssm import parameter_store

In [ ]:
pm = parameter_store(region)
strPrefix = pm.get_params(key="PREFIX")

In [ ]:
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-PREP"]), value=strEcrRepositoryUri_prep, overwrite=True)
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-TR"]), value=strEcrRepositoryUri_tr, overwrite=True)
pm.put_params(key="-".join([strPrefix, "IMAGE-URI-INF"]), value=strEcrRepositoryUri_inf, overwrite=True)

In [ ]:
print ("IMAGE-URI-PREP: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-PREP"])))
print ("IMAGE-URI-TR: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-TR"])))
print ("IMAGE-URI-INF: ", pm.get_params(key="-".join([strPrefix, "IMAGE-URI-INF"])))